# Abordagem 1

Usando a abordagem 1 para gerar templates com foco em templates positivos e negativos. Uma possível aplicação seria testar a capacidade linguística *Vocabullary* com o teste **MFT**.

As etapas desta abordagem são:

1. Rankear as palavras das instâncias completas
2. Quebrar as instâncias em sentenças
3. Filtrar as sentenças que contêm ao menos uma das palavras mais bem rankeadas na etapa anterior
4. Filtrar as sentenças com palavras relevantes (adjetivos ou verbos)
5. Classificar as sentenças usando o *Oráculo*
6. Filtrar as sentenças classificadas de forma unânime
7. Substituir as palavras relevantes por máscaras

In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../')

## Carregando o dataset, o modelo alvo e os modelos auxiliares

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

imdb_df = pd.read_csv('./data/imdb_sampled/data-100samples.csv')
imdb_df.head(5)

,label,text,words
0,1,"Christian Duguay directed this tidy little espionage thriller early in his career. It plays on TV pretty regularly, albeit with some terrific scenes of violence and sex unfortunately trimmed. I finally got around to seeing the theatrical version on a $3 tape from the local video store. Naval officer Aidan Quinn is recruited to impersonate the notorious Carlos the Jackal, and gets a little too caught up in the role. Donald Sutherland Ben Kingsley play Quinn's superiors, with Sutherland a true zealot and Kingsley as the more level-headed one. The first half of this fun flick shows Quinn being trained and indoctrinated. The second half has him out in the field, making love to the Jackal's woman and shooting it out with sundry enemies. The idea is to make the Jackal look like a turncoat to the Russians, and let them take care of the world's most notorious assassin. Things don't exactly play out as planned. At times, I almost expected the cast to break out laughing at some of the corny dialogue, but they all play it very straight. In the end, this is one terrific little thriller that deserves your attention. The Jackal's former mistress teaching the highly proper and very married Quinn to rough her up, lick blood from her face, and then go down on her, alone is worth the price of admission.",227
1,1,"New Yorkers contemporaneous with this film will recall how reflective of its time it is and how well cast and crew captured America, New York City of that era.<br /><br />Norman Wexler's script delineates the different worlds the various sub groupings live in and Avildsen's direction brings out phenomenal performances all around. Peter Boyle's prodigious talent is on display as never before nor since. Clearly it is the best character portrayal the always likable Dennis Patrick ever accomplished.<br /><br />What I will always remember about JOE is the feeling of having been in a virtual state of shock coming out of the theater. Knowing that what the screen portrayed was seething under the surface in neighborhoods throughout the five boroughs of the City of New York.<br /><br />This film needs to be remembered.",133
2,0,"I love oddball animation, I love a lot of Asian films, but I didn't love this particular product of Japan. The Fuccons are supposedly an American family (they're all mannequins) who have moved to Japan, and they're somewhat a 50's sitcom type family, with slightly more modern sensibilities at times. The DVD features several very short episodes (like less than 5 minutes each?) and I did not find it to be either funny or entertaining, not even in a weird way. I'm not sure what the appeal is of this. I did pick up on some satire here and there, gosh, who wouldn't, but satire is usually somewhat humorous, isn't it? And nothing I saw or heard rated even a little smirk. I picked this up used and it certainly SOUNDED appealing, but I guess either I'm missing the point or it's just plain LAME. The box even says it's Fuccon hilarious, right there on the front, but I beg to differ. 2 out of 10.",166
3,1,"I have seen this film probably a dozen times since it was originally released theatrically. Anyone who calls this movie trash or horrible just doesn't understand action films or recognize a good one. Perhaps to some the incidents and outcomes may seem far fetched, but in my opinion screenwriter Shane Black ( Lethal Weapon/ Kiss Kiss Bang Bang) crafted one of the most well thought out action adventures you will ever come across. Over the top or not this film flows like clockwork and the action just keeps coming. The final action sequence is one of the best I have ever seen in any film. The cast in this film crackles. Genna Davis gave a tremendous performance and its a damn shame there was never a ""LKG"" sequel. Samuel L. Jackson is hilarious as her sidekick Mitch a down on his luck private eye trying to help her discover her lost past and make a few bucks. If Baffles me how anyone could not like this film. I

In [3]:
import re
import numpy as np
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def pre_proccess(text):
    text = text.lower()
    text = re.sub('["\',!-.:-@0-9/]()', ' ', text)
    return text

# Wrapper to adapt output format
class SentimentAnalisysModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __predict(self, text_input):
        text_preprocessed = pre_proccess(text_input)
        tokenized = self.tokenizer(text_preprocessed, padding=True, truncation=True, max_length=512, 
                                    add_special_tokens = True, return_tensors="pt")
        
        tensor_logits = self.model(**tokenized)
        prob = softmax(tensor_logits[0]).detach().numpy()
        pred = np.argmax(prob)
        
        return pred, prob
    
    def predict_label(self, text_inputs):
        return self.predict(text_inputs)[0]
        
    def predict_proba(self, text_inputs):
        return self.predict(text_inputs)[1]
        
    def predict(self, text_inputs):
        if isinstance(text_inputs, str):
            text_inputs = [text_inputs]
        
        preds = []
        probs = []

        for text_input in text_inputs:
            pred, prob = self.__predict(text_input)
            preds.append(pred)
            probs.append(prob[0])

        return np.array(preds), np.array(probs) # ([0, 1], [[0.99, 0.01], [0.03, 0.97]])

# Auxiliar function to load and wrap a model from Hugging Face
def load_model(model_name):
    print(f'Loading model {model_name}...')
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return SentimentAnalisysModelWrapper(model, tokenizer)

# Hugging Face hosted model names 
imdb_models = {
    'bert': 'textattack/bert-base-uncased-imdb', 
    'albert': 'textattack/albert-base-v2-imdb', 
    'distilbert': 'textattack/distilbert-base-uncased-imdb', 
    'roberta': 'textattack/roberta-base-imdb', 
    'xlnet': 'textattack/xlnet-base-cased-imdb'
}

In [4]:
m1 = load_model(imdb_models['albert'])
m2 = load_model(imdb_models['distilbert'])
m3 = load_model(imdb_models['roberta'])
m4 = load_model(imdb_models['xlnet'])

# Models to be used as oracle
models = [m1, m2, m3, m4]
# Target model
model = load_model(imdb_models['bert'])

Loading model textattack/albert-base-v2-imdb...
Loading model textattack/distilbert-base-uncased-imdb...
Loading model textattack/roberta-base-imdb...


Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model textattack/xlnet-base-cased-imdb...
Loading model textattack/bert-base-uncased-imdb...


# Gerando os templates
O método de rankeamento das palavras usado no PosNegTemplateGenerator é o Replace-1 Score

In [5]:
from template_generator.tasks.sentiment_analisys import PosNegTemplateGeneratorApp1

tg = PosNegTemplateGeneratorApp1(model, models)

### Número inicial de instâncias: 5

In [6]:
# Sampling instances
np.random.seed(220)
n_instances = 5
df_sampled = imdb_df.sample(n_instances)

instances = [x for x in df_sampled['text'].values]

In [7]:
templates = tg.generate_templates(instances)

Ranking words using Replace-1 Score...


/tmp/ipykernel_12054/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Converting texts to sentences...
:: 40 sentences were generated.
Filtering instances by contaning ranked words...
:: 16 sentences remaining.
Filtering instances by relevant words...
:: 0 sentences remaining.
Predicting inputs...
:: Sentence predictions done.


#### Tempo de execução para 5 instâncias: 6m 8.6s
filipe: 2m 20.5s

In [8]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text


In [9]:
tg.lexicons

{'pos_verb': [], 'neg_verb': [], 'pos_adj': [], 'neg_adj': []}

### Número inicial de instâncias: 100

In [10]:
# Using all 100 instances
instances = [x for x in imdb_df['text'].values]

In [11]:
# 60 min 50.1s
tg = PosNegTemplateGeneratorApp1(model, models)

templates = tg.generate_templates(instances)

Ranking words using Replace-1 Score...


/tmp/ipykernel_12054/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Converting texts to sentences...
:: 852 sentences were generated.
Filtering instances by contaning ranked words...
:: 371 sentences remaining.
Filtering instances by relevant words...
:: 24 sentences remaining.
Predicting inputs...
:: Sentence predictions done.


#### Tempo de execução para 100 instâncias: 123m 6.7s

In [20]:
df_templates = tg.to_dataframe()
df_templates.insert(0, "template_index", df_templates.index.map(lambda x: int(x)+1))
df_templates

,template_index,label,original_text,masked_text,template_text
0,1,0,Things don't exactly play out as planned.,Things do n't exactly {mask} out as {mask} .,Things do n't exactly {neg_verb} out as {neg_verb} .
1,2,1,"Don't mind what this socially retarded person above says, this show is hilarious.","Do n't mind what this socially retarded person above {mask} , this show is {mask} .","Do n't mind what this socially retarded person above {neg_verb} , this show is {pos_adj} ."
2,3,1,This movie is trying to make us all evaluate our lives and take off the rose colored glasses that most of us view the world through.<br /><br />Great movie.,This movie is trying to make us all {mask} our lives and take off the rose colored glasses that most of us view the world through.<br /><br {mask} movie .,This movie is trying to make us all {neg_verb} our lives and take off the rose colored glasses that most of us view the world through.<br /><br {pos_adj} movie .
3,4,0,Richard Roundtree has little more than a cameo as the police chief.,Richard Roundtree {mask} {mask} more than a cameo as the police chief .,Richard Roundtree {neg_verb} {neg_adj} more than a cameo as the police chief .
4,5,1,the b&w version was more authentic.. but this is a good film beautifully photographed in color & panavision...,the b&w version was more authentic .. but this is a {mask} film beautifully {mask} in color & panavision ...,the b&w version was more authentic .. but this is a {pos_adj} film beautifully {pos_verb} in color & panavision ...
5,6,0,"Then tonight I decided to see it, the film was quite different to what I had expected and I didn't find any humour in it all I saw was that it was a bleak look at people dealing with love relationships and sexual orientation and I didn't really see the psycho killer plot really having a point except to add tension to the end of the film.","Then tonight I decided to see it , the film was quite {mask} to what I had {mask} and I did n't find any humour in it all I saw was that it was a bleak look at people dealing with love relationships and sexual orientation and I did n't really see the psycho killer plot really having a point except to add tension to the end of the film .","Then tonight I decided to see it , the film was quite {pos_adj} to what I had {neg_verb} and I did n't find any humour in it all I saw was that it was a bleak look at people dealing with love relationships and sexual orientation and I did n't really see the psycho killer plot really having a point except to add tension to the end of the film ."
6,7,1,I felt that the person playing the lesbian woman did a great job.,I {mask} that the person playing the lesbian woman did a {mask} job .,I {pos_verb} that the person playing the lesbian woman did a {pos_adj} job .
7,8,0,"Barbet Schroeder (at one time such a promising director with films like ""Barfly"" that amounted to pretty much nothing in the American market) directs well enough - I honestly thought the script was the culprit here...it's just a big mess.<br /><br />The film ultimately wastes a lot of good material, good actors and a good director - all because of a faulty script.","Barbet Schroeder ( at one time such a promising director with films like "" Barfly "" that amounted to pretty much nothing in the American market ) directs well enough - I honestly thought the script was the culprit here ... it 's just a big mess.<br /><br />The film ultimately {mask} a lot of good material , good actors and a good director - all because of a {mask} script .","Barbet Schroeder ( at one time such a promising director with films like "" Barfly "" that amounted to pretty much nothing in the American market ) directs well enough - I honestly thought the script was the culprit here ... it 's just a big mess.<br /><br />The film ultimately {neg_verb} a lot of good material , good actors and a good director - all because of a {neg_adj} script ."
8,9,1,"Though, in some ways, it leaves little to the imagination, and shows 

In [21]:
df_templates.to_csv("generated_templates/generated_templates_approach1.csv")


In [13]:
tg.lexicons

{'pos_verb': ['photographed', 'witness', 'felt', 'agree'],
 'neg_verb': ['wastes',
  'supposed',
  'continues',
  'play',
  'looked',
  'evaluate',
  'bored',
  'expected',
  'blabbering',
  'has',
  'planned',
  'crawls',
  'takes',
  'looks',
  'says'],
 'pos_adj': ['bizarre',
  'natured',
  'great',
  'lively',
  'different',
  'outstanding',
  '/>Great',
  'good',
  'hilarious',
  'goofy',
  'picky',
  'alluring'],
 'neg_adj': ['low',
  'little',
  'rebellious',
  'nitpicky',
  'appalling',
  'incestuous',
  'drab',
  'new',
  'Other',
  'unbearable',
  'worst',
  'bad',
  'pyroclastic',
  'faulty']}

# Usando os templates gerados pelo TemplateGenerator no CheckList

In [14]:
from checklist.editor import Editor
from checklist.test_suite import TestSuite
from checklist.test_types import MFT

In [15]:
lexicons = tg.lexicons
templates = tg.template_texts
masked = tg.masked_texts
labels = [sent.prediction.label for sent in tg.sentences]

editor = Editor()
editor.add_lexicon('pos_verb', lexicons['pos_verb'])
editor.add_lexicon('neg_verb', lexicons['neg_verb'])
editor.add_lexicon('pos_adj', lexicons['pos_adj'])
editor.add_lexicon('neg_adj', lexicons['neg_adj'])

suite = TestSuite()

In [16]:
for template, label, i in zip(templates, labels, range(len(templates))):
    t = editor.template(template, remove_duplicates=True, labels=int(label))

    suite.add(MFT(
        data=t.data,
        labels=label,
        capability="Vocabullary", 
        name=f"Test: MFT with vocabullary - template{i+1}",
        description="Checking if the model can handle vocabullary"))

In [17]:
suite.run(model.predict, overwrite=True)
suite.save('./suites/posneg-approach1.suite')

Running Test: MFT with vocabullary - template1
Predicting 15 examples


/tmp/ipykernel_12054/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Running Test: MFT with vocabullary - template2
Predicting 180 examples
Running Test: MFT with vocabullary - template3
Predicting 180 examples
Running Test: MFT with vocabullary - template4
Predicting 210 examples
Running Test: MFT with vocabullary - template5
Predicting 48 examples
Running Test: MFT with vocabullary - template6
Predicting 180 examples
Running Test: MFT with vocabullary - template7
Predicting 48 examples
Running Test: MFT with vocabullary - template8
Predicting 210 examples
Running Test: MFT with vocabullary - template9
Predicting 168 examples
Running Test: MFT with vocabullary - template10
Predicting 210 examples
Running Test: MFT with vocabullary - template11
Predicting 56 examples
Running Test: MFT with vocabullary - template12
Predicting 12 examples
Running Test: MFT with vocabullary - template13
Predicting 56 examples
Running Test: MFT with vocabullary - template14
Predicting 168 examples
Running Test: MFT with vocabullary - template15
Predicting 14 examples
Runnin

# Carregando suite de teste

In [18]:
from checklist.test_suite import TestSuite
suite = TestSuite.from_file('./suites/posneg-approach1.suite')

suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Test: MFT with vocab…

In [22]:
passed = 0
failed = 0
for test_name in suite.tests:
    table = suite.visual_summary_by_test(test_name)
    
    failed += table.stats['nfailed']    
    passed += table.stats['npassed']
    assert table.stats['nfailed'] + table.stats['npassed'] == len(table.filtered_testcases)

print(f"{failed = } ({(failed/(passed+failed))*100:.2f}%)")
print(f"{passed = } ({(passed/(passed+failed))*100:.2f}%)")
print(f"total = {passed+failed}")
print("templates:", len(suite.tests))

failed = 815 (25.53%)
passed = 2377 (74.47%)
total = 3192
templates: 24


In [19]:
table = suite.visual_summary_by_test('Test: MFT with vocabullary - template5')

failed = table.candidate_testcases
tests = table.filtered_testcases

for item in tests:
    if not item in failed:
        print(item['examples'][0]['new']['text'])

the b&w version was more authentic .. but this is a bizarre film beautifully photographed in color & panavision ...
the b&w version was more authentic .. but this is a bizarre film beautifully witness in color & panavision ...
the b&w version was more authentic .. but this is a bizarre film beautifully felt in color & panavision ...
the b&w version was more authentic .. but this is a bizarre film beautifully agree in color & panavision ...
the b&w version was more authentic .. but this is a natured film beautifully photographed in color & panavision ...
the b&w version was more authentic .. but this is a natured film beautifully witness in color & panavision ...
the b&w version was more authentic .. but this is a natured film beautifully felt in color & panavision ...
the b&w version was more authentic .. but this is a natured film beautifully agree in color & panavision ...
the b&w version was more authentic .. but this is a great film beautifully photographed in color & panavision ..